# Housing rent prices and venues data analysis of Frankfurt

#### Import all the packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip -q install folium
import folium # map rendering library

!pip -q install plotly

# to retrieve the districts coordinates
import urllib.parse

print('Libraries imported.')

Libraries imported.


## Data preparation

### Retrieve list of Frankfurt's districts from Wikipedia's page

In [2]:
# Webpage url (old link as suggested in the Discussion Forums)                                                                                                             
url = 'https://de.wikipedia.org/wiki/Liste_der_Stadtteile_von_Frankfurt_am_Main'

# Extract tables using pandas
dfs = pd.read_html(url,header=[0],flavor='bs4')

# Get first table                                                                                                           
df = dfs[0]

df

,Nr.,Stadtteil,Fläche[3]in km²,Einwohner,Weiblich,Männlich,Deutsche,Ausländer,Ausländerin Prozent,Einwohnerje km²,Ortsbezirk,Stadtgebietseit,Vorherige Zugehörigkeit
0,1,Altstadt,506,4.218,2.065,2.153,2.669,1.549,367,8336,01 Innenstadt I,1866[Anm. 1],Freie Stadt Frankfurt
1,2,Innenstadt,1491,6.599,3.112,3.487,3.539,3.060,464,4426,01 Innenstadt I,1866[Anm. 2],Freie Stadt Frankfurt
2,3,Bahnhofsviertel,542,3.552,1.321,2.231,1.706,1.846,52,6554,01 Innenstadt I,1866[Anm. 3],Freie Stadt Frankfurt
3,4,Westend-Süd,2497,19.314,9.839,9.475,14.006,5.308,275,7735,02 Innenstadt II,1866[Anm. 3],Freie Stadt Frankfurt
4,5,Westend-Nord,1632,10.373,5.391,4.982,7.366,3.007,29,6356,02 Innenstadt II,1866[Anm. 3],Freie Stadt Frankfurt
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,44,Harheim,4837,5.234,2.664,2.570,4.396,838,16,1082,14 Harheim,1972[6],Landkreis Friedberg
43,45,Nieder-Eschbach,6348,11.518,5.932,5.586,8.756,2.762,24,1814,15 Nieder-Eschbach,1972[6],Landkreis Friedberg
44,46,Bergen-Enkheim,12601,17.941,9.240,8.701,14.321,3.620,202,1424,16 Bergen-Enkheim,1977[8],Main-Kinzig-Kreis
45,47,Frankfurter Berg,2400,8.168,4.110,4.058,5.987,2.181,267,3403,10 Nord-Ost,1910[Anm. 15],Landkreis Frankfurt[Anm. 4]


In [3]:
df_districts = df.iloc[0:-1,1:8].reset_index(drop=True) # Remove the last row which is the total
df_districts.columns = ['District', 'Surface_km2',"population", "female_pop", "male_pop", "german_pop", "foreign_pop"]

#Fix Flughafen value
df_districts.loc[df_districts.District == "Flughafen", ["population","female_pop", "male_pop", "german_pop", "foreign_pop"]] = df_districts.loc[df_districts.District == "Sachsenhausen-Süd", ["population","female_pop", "male_pop", "german_pop", "foreign_pop"]].values.tolist()

df_districts[["population","female_pop","male_pop","german_pop","foreign_pop"]] = df_districts[["population","female_pop","male_pop","german_pop","foreign_pop"]].apply(lambda x: x.str.replace('.',''))
df_districts = df_districts.astype({'population': 'int64', 'female_pop': 'int64', 'male_pop': 'int64', 'german_pop': 'int64', 'foreign_pop': 'int64'})

df_districts

,District,Surface_km2,population,female_pop,male_pop,german_pop,foreign_pop
0,Altstadt,506,4218,2065,2153,2669,1549
1,Innenstadt,1491,6599,3112,3487,3539,3060
2,Bahnhofsviertel,542,3552,1321,2231,1706,1846
3,Westend-Süd,2497,19314,9839,9475,14006,5308
4,Westend-Nord,1632,10373,5391,4982,7366,3007
...,...,...,...,...,...,...,...
41,Kalbach-Riedberg,6580,21795,11047,10748,16683,5112
42,Harheim,4837,5234,2664,2570,4396,838
43,Nieder-Eschbach,6348,11518,5932,5586,8756,2762
44,Bergen-Enkheim,12601,17941,9240,8701,14321,3620


## Get the coordinates of the districts

In [22]:
df_districts["Latitude"] = 0.00000
df_districts["Longitude"] = 0.00000
df_districts["Coordinates"] = np.empty((len(df_districts), 0)).tolist()

for row, district in enumerate(df_districts["District"]):
    downloaded = False
    
    while downloaded == False:
        try:
            print("Checking "+district)
            address = district+', Frankfurt am Main, Germany'
            url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?polygon_geojson=1&format=json'
            #print(url)

            response = requests.get(url).json()
            df_districts["Latitude"][row] = round(float((response[0]["lat"])), 5)
            df_districts["Longitude"][row] = round(float((response[0]["lon"])), 5)
            df_districts["Coordinates"][row] = response[0]["geojson"]['coordinates']
            downloaded = True
        except:
              print("Error with district download - trying again")

#df_districts

Checking Altstadt
Checking Innenstadt


<ipython-input-22-687ef620f061>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_districts["Latitude"][row] = round(float((response[0]["lat"])), 5)
<ipython-input-22-687ef620f061>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_districts["Longitude"][row] = round(float((response[0]["lon"])), 5)
<ipython-input-22-687ef620f061>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_districts["Coordinates"][row] = response[0]["geoj

Checking Bahnhofsviertel
Checking Westend-Süd
Checking Westend-Nord
Checking Nordend-West
Checking Nordend-Ost
Checking Ostend
Checking Bornheim
Checking Gutleutviertel
Checking Gallus
Checking Bockenheim
Checking Sachsenhausen-Nord
Checking Sachsenhausen-Süd
Checking Flughafen
Checking Oberrad
Checking Niederrad
Checking Schwanheim
Checking Griesheim
Checking Rödelheim
Checking Hausen
Checking Praunheim
Checking Heddernheim
Checking Niederursel
Checking Ginnheim
Checking Dornbusch
Checking Eschersheim
Checking Eckenheim
Checking Preungesheim
Checking Bonames
Checking Berkersheim
Checking Riederwald
Checking Seckbach
Checking Fechenheim
Checking Höchst
Checking Nied
Checking Sindlingen
Checking Zeilsheim
Checking Unterliederbach
Checking Sossenheim
Checking Nieder-Erlenbach
Checking Kalbach-Riedberg
Checking Harheim
Checking Nieder-Eschbach
Checking Bergen-Enkheim
Checking Frankfurter Berg


In [23]:
df_districts

,District,Surface_km2,population,female_pop,male_pop,german_pop,foreign_pop,Latitude,Longitude,Coordinates
0,Altstadt,506,4218,2065,2153,2669,1549,50.11044,8.68235,"[[[8.6748666, 50.1092226], [8.6757349, 50.1075..."
1,Innenstadt,1491,6599,3112,3487,3539,3060,50.11456,8.68359,"[[[8.6685139, 50.1132235], [8.6690588, 50.1125..."
2,Bahnhofsviertel,542,3552,1321,2231,1706,1846,50.10841,8.66815,"[[[8.6605492, 50.1095394], [8.6625518, 50.1085..."
3,Westend-Süd,2497,19314,9839,9475,14006,5308,50.11524,8.66227,"[[[8.6433346, 50.1140438], [8.6434858, 50.1091..."
4,Westend-Nord,1632,10373,5391,4982,7366,3007,50.12636,8.66792,"[[[8.6558043, 50.1266272], [8.6564224, 50.1259..."
...,...,...,...,...,...,...,...,...,...,...
41,Kalbach-Riedberg,6580,21795,11047,10748,16683,5112,50.18628,8.63905,"[[[8.6140631, 50.1767957], [8.6172061, 50.1746..."
42,Harheim,4837,5234,2664,2570,4396,838,50.18229,8.69297,"[[[8.6711429, 50.1865714], [8.6727365, 50.1856..."
43,Nieder-Eschbach,6348,11518,5932,5586,8756,2762,50.20173,8.66671,"[[[8.6436283, 50.2023429], [8.6446848, 50.2019..."
44,Bergen-Enkheim,12601,17941,9240,8701,14321,3620,50.15801,8.76204,"[[[8.735863, 50.1629854], [8.7394536, 50.15992..."


## Download data on Rental price index 

In [5]:
# Webpage url (old link as suggested in the Discussion Forums)                                                                                                             
url = 'https://www.miet-check.de/stadtteile_uebersicht.php?stadt=Frankfurt%20am%20Main'

# Extract tables using pandas
dfs = pd.read_html(url,header=[0],flavor='bs4')

# Get first table                                                                                                           
df = dfs[0]

district_rent_price = pd.DataFrame(columns =["District", "Rent_per_sqm"])

district_rent_price = district_rent_price.append(df[["Stadtteile in","Stadtteile in.1"]].rename(columns={"Stadtteile in": "District", "Stadtteile in.1": "Rent_per_sqm"}), ignore_index=True)
district_rent_price = district_rent_price.append(df[["Stadtteile in.2","Stadtteile in.3"]].rename(columns={"Stadtteile in.2": "District", "Stadtteile in.3": "Rent_per_sqm"}), ignore_index=True)
district_rent_price = district_rent_price.append(df[["Stadtteile in.4","Stadtteile in.5"]].rename(columns={"Stadtteile in.4": "District", "Stadtteile in.5": "Rent_per_sqm"}), ignore_index=True)
district_rent_price["Rent_per_sqm"] = district_rent_price["Rent_per_sqm"].str.replace(",", ".").str.replace(" €", "").astype(float)

pd.set_option('display.max_rows', None)

#Price split for Sachsenhausen Nord and Sud not available 
district_rent_price.loc[district_rent_price.District == "Sachsenhausen", "District"] = "Sachsenhausen-Süd"
district_rent_price.loc[45] = ["Sachsenhausen-Nord", float(district_rent_price.loc[district_rent_price.District == "Sachsenhausen-Süd"].Rent_per_sqm)]
#Altstadt price same as Innenstadt
district_rent_price.loc[46] = ["Altstadt", float(district_rent_price.loc[district_rent_price.District == "Innenstadt"].Rent_per_sqm)]

# Merge back to original df
df_districts = pd.merge(df_districts, district_rent_price, on="District")

In [28]:
import geopandas as gpd 
from shapely.geometry import Polygon

df_districts['geometry'] = gdf.apply(lambda row:Polygon(eval(row['coordinates'])), axis=1)

^C


ModuleNotFoundError: No module named 'geopandas'


Examining jupyter_console: 100%|#########9| 313/314 [02:18<00:00,  2.42it/s]
                                                                            

Determining conflicts:   0%|          | 0/314 [00:00<?, ?it/s]
Examining conflict for paramiko spyder bcrypt prometheus_client anaconda:   0%|          | 0/314 [00:00<?, ?it/s]
Examining conflict for argh patsy fastcache partd bcrypt ipython cython python-language-server win_inet_pton urllib3 wheel pytest pyreadline joblib backports.shutil_get_terminal_size backports.functools_lru_cache gevent matplotlib-base diff-match-patch toml pytz openpyxl jupyterlab_pygments sphinxcontrib-htmlhelp statsmodels get_terminal_size pathtools traitlets bkcharts spyder decorator flask python_abi defusedxml simplegeneric bokeh xlsxwriter packaging intervaltree anaconda flake8 pep8 paramiko ipykernel python-libarchive-c astroid rtree mistune imageio jupyterlab sphinxcontrib-jsmath wcwidth olefile importlib-metadata numba pywinpty click itsdangerous zip

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed


sphinxcontrib-devhelp -> python[version='>=3.5'] -> openssl[version='>=1.1.1a,<1.1.2a|>=1.1.1b,<1.1.2a|>=1.1.1c,<1.1.2a|>=1.1.1d,<1.1.2a|>=1.1.1e,<1.1.2a|>=1.1.1f,<1.1.2a|>=1.1.1g,<1.1.2a|>=1.1.1h,<1.1.2a|>=1.1.1i,<1.1.2a|>=1.1.1j,<1.1.2a']
jupyter -> python[version='>=3.9,<3.10.0a0'] -> openssl[version='>=1.1.1a,<1.1.2a|>=1.1.1b,<1.1.2a|>=1.1.1c,<1.1.2a|>=1.1.1d,<1.1.2a|>=1.1.1e,<1.1.2a|>=1.1.1f,<1.1.2a|>=1.1.1g,<1.1.2a|>=1.1.1h,<1.1.2a|>=1.1.1i,<1.1.2a|>=1.1.1j,<1.1.2a']
anaconda-navigator -> python[version='>=3.9,<3.10.0a0'] -> openssl[version='>=1.1.1a,<1.1.2a|>=1.1.1b,<1.1.2a|>=1.1.1c,<1.1.2a|>=1.1.1d,<1.1.2a|>=1.1.1e,<1.1.2a|>=1.1.1f,<1.1.2a|>=1.1.1g,<1.1.2a|>=1.1.1h,<1.1.2a|>=1.1.1i,<1.1.2a|>=1.1.1j,<1.1.2a']
path.py -> python[version='>=3.5'] -> openssl[version='>=1.1.1a,<1.1.2a|>=1.1.1b,<1.1.2a|>=1.1.1c,<1.1.2a|>=1.1.1d,<1.1.2a|>=1.1.1e,<1.1.2a|>=1.1.1f,<1.1.2a|>=1.1.1g,<1.1.2a|>=1.1.1h,<1.1.2a|>=1.1.1i,<1.1.2a|>=1.1.1j,<1.1.2a']
msgpack-python -> python[version='>=3.8,<3.9.0a

## Create a map of Frankfurt with neighborhoods superimposed on top


In [7]:
address = 'Frankfurt am Main, DE'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Frankfurt are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Frankfurt are 50.1106444, 8.6820917.


In [8]:
import folium
from urllib.request import urlopen
import json

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# file name - file is located in the working directory
with urlopen('https://offenedaten.frankfurt.de/dataset/85b38876-729c-4a78-910c-a52d5c6df8d2/resource/84dff094-ab75-431f-8c64-39606672f1da/download/ffmstadtteilewahlen.geojson') as response:
    districts = json.load(response)
    
communities_geo = districts # geojson file

# create a plain Frankfur map
communities_map = folium.Map(location=[latitude, longitude], zoom_start=11)

#generate choropleth map 
folium.Choropleth(
    geo_data=communities_geo,
    data=df_districts[['District', "Rent_per_sqm"]],
    columns=['District', 'Rent_per_sqm'],
    key_on='feature.properties.STTLNAME',
    fill_color='PuRd', 
    fill_opacity=1, 
    line_opacity=1,
    legend_name='Rent per sqm',
    smooth_factor=0).add_to(communities_map)

# display map
#communities_map
# add markers to map
for lat, lng, district in zip(df_districts['Latitude'], df_districts['Longitude'], df_districts['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(communities_map)  


style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    districts,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['STTLNAME'],
        aliases=['Neighborhood'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

communities_map.add_child(NIL)
#communities_map.keep_in_front(NIL)
folium.LayerControl().add_to(communities_map)
communities_map